# Step 12: Cosmos DB Memory Integration

This notebook demonstrates persistent chat history using Azure Cosmos DB for MongoDB.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

**Note:** This example requires Azure Cosmos DB for MongoDB connection. Make sure your environment variables are configured.

In [ ]:
import asyncio
from dataclasses import dataclass
from typing import Annotated

from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.data.vector import (
    VectorStoreField,
    vectorstoremodel,
    FieldTypes,
)
from semantic_kernel.connectors.azure_cosmos_db import (
    CosmosMongoStore,
    CosmosMongoCollection,
)
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)

## Define the Chat History Data Model

In [ ]:
@vectorstoremodel
@dataclass
class ChatHistoryModel:
    """Data model for storing chat history in Cosmos DB."""
    session_id: Annotated[str, VectorStoreField(field_type=FieldTypes.KEY)]
    user_id: Annotated[str, VectorStoreField(is_indexed=True)]
    messages: Annotated[list[dict[str, str]], VectorStoreField(is_indexed=True)]

## Create Custom ChatHistory Class

In [ ]:
class ChatHistoryInCosmosDB(ChatHistory):
    """Extends ChatHistory to persist messages in Azure Cosmos DB."""
    session_id: str
    user_id: str
    store: CosmosMongoStore
    collection: CosmosMongoCollection[str, ChatHistoryModel] | None = None

    async def create_collection(self, collection_name: str) -> None:
        self.collection = self.store.get_collection(
            ChatHistoryModel, collection_name=collection_name
        )

    async def store_messages(self) -> None:
        if self.collection:
            try:
                await self.collection.upsert(
                    ChatHistoryModel(
                        session_id=self.session_id,
                        user_id=self.user_id,
                        messages=[msg.model_dump() for msg in self.messages],
                    )
                )
            except Exception as e:
                print(f"Error: {e}")

    async def read_messages(self) -> None:
        self.messages.clear()
        if self.collection:
            record = await self.collection.get(self.session_id)
            if record:
                for message in record.messages:
                    self.messages.append(ChatMessageContent.model_validate(message))

## Setup Kernel and Services

In [ ]:
kernel = Kernel()
kernel.add_plugin(MathPlugin(), plugin_name="math")
kernel.add_plugin(TimePlugin(), plugin_name="time")

chat_service = AzureChatCompletion(service_id="default")
request_settings = AzureChatPromptExecutionSettings(service_id="default")
request_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(
    filters={"excluded_plugins": ["ChatBot"]}
)

kernel.add_service(chat_service)

## Define Chat Loop

In [ ]:
async def chat(history: ChatHistoryInCosmosDB) -> bool:
    await history.read_messages()
    print(f"Loaded {len(history.messages)} messages from Cosmos DB")

    if len(history.messages) == 0:
        history.add_system_message(
            "You are RK Bot, a curious assistant that helps people figure out what they need."
        )
        history.add_user_message("Hi there, who are you?")
        history.add_assistant_message("I'm RK Bot! Ask me anything.")

    try:
        user_input = input("User:> ")
    except (KeyboardInterrupt, EOFError):
        print("\nExiting chat...")
        return False

    if user_input.lower().strip() == "exit":
        return False

    history.add_user_message(user_input)

    result = await chat_service.get_chat_message_content(
        history, request_settings, kernel=kernel
    )

    if result:
        print(f"RK Bot:> {result}")
        history.add_message(result)

    print(f"Storing {len(history.messages)} messages to Cosmos DB...")
    await history.store_messages()

    return True

## Run the Chat

Execute this to start the chat session with persistent memory.

In [ ]:
async def main():
    session_id = "session123"
    delete_when_done = False

    async with CosmosMongoStore() as store:
        history = ChatHistoryInCosmosDB(
            store=store, session_id=session_id, user_id="user"
        )
        await history.create_collection("chat_memory")

        print("Welcome to the RK Bot assistant! Type 'exit' to quit.")
        print("Try asking: What time is it? or What is 4 * 5?")

        chatting = True
        while chatting:
            chatting = await chat(history)

        if delete_when_done and history.collection:
            await history.collection.delete_collection()


In [ ]:
await main()